# Abordagem 1

Usando a abordagem 1 para gerar templates com foco em templates positivos e negativos. Uma possível aplicação seria testar a capacidade linguística *Vocabullary* com o teste **MFT**.

As etapas desta abordagem são:

1. Rankear as palavras das instâncias completas
2. Quebrar as instâncias em sentenças
3. Filtrar as sentenças que contêm ao menos uma das palavras mais bem rankeadas na etapa anterior
4. Filtrar as sentenças com palavras relevantes (adjetivos ou verbos)
5. Classificar as sentenças usando o *Oráculo*
6. Filtrar as sentenças classificadas de forma unânime
7. Substituir as palavras relevantes por máscaras

In [1]:
%config Completer.use_jedi = False
import sys
sys.path.append('../')

## Carregando o dataset, o modelo alvo e os modelos auxiliares

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
dataset.set_format("pandas")
df = dataset["test"].shuffle(seed=42)[:100]
df

,text,label
0,"unpretentious , charming , quirky , original",1
1,"a film really has to be exceptional to justify a three hour running time , and this isn't .",0
2,working from a surprisingly sensitive script co-written by gianni romoli . . . ozpetek avoids most of the pitfalls you'd expect in such a potentially sudsy set-up .,1
3,"it may not be particularly innovative , but the film's crisp , unaffected style and air of gentle longing make it unexpectedly rewarding .",1
4,"such a premise is ripe for all manner of lunacy , but kaufman and gondry rarely seem sure of where it should go .",0
...,...,...
95,"ice age is the first computer-generated feature cartoon to feel like other movies , and that makes for some glacial pacing early on .",0
96,there's no denying that burns is a filmmaker with a bright future ahead of him .,1
97,it collapses when mr . taylor tries to shift the tone to a thriller's rush .,0
98,"there's a great deal of corny dialogue and preposterous moments . and yet , it still works .",1


In [3]:
df["label"].value_counts()

label
0    51
1    49
Name: count, dtype: int64

In [4]:
import re
import numpy as np
from torch.nn.functional import softmax
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def pre_proccess(text):
    text = text.lower()
    text = re.sub('["\',!-.:-@0-9/]()', ' ', text)
    return text

# Wrapper to adapt output format
class SentimentAnalisysModelWrapper:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        
    def __predict(self, text_input):
        text_preprocessed = pre_proccess(text_input)
        tokenized = self.tokenizer(text_preprocessed, padding=True, truncation=True, max_length=512, 
                                    add_special_tokens = True, return_tensors="pt")
        
        tensor_logits = self.model(**tokenized)
        prob = softmax(tensor_logits[0]).detach().numpy()
        pred = np.argmax(prob)
        
        return pred, prob
    
    def predict_label(self, text_inputs):
        return self.predict(text_inputs)[0]
        
    def predict_proba(self, text_inputs):
        return self.predict(text_inputs)[1]
        
    def predict(self, text_inputs):
        if isinstance(text_inputs, str):
            text_inputs = [text_inputs]
        
        preds = []
        probs = []

        for text_input in text_inputs:
            pred, prob = self.__predict(text_input)
            preds.append(pred)
            probs.append(prob[0])

        return np.array(preds), np.array(probs) # ([0, 1], [[0.99, 0.01], [0.03, 0.97]])

# Auxiliar function to load and wrap a model from Hugging Face
def load_model(model_name):
    print(f'Loading model {model_name}...')
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    return SentimentAnalisysModelWrapper(model, tokenizer)

# Hugging Face hosted model names 
rotten_tomatoes_models = {
    'bert': 'textattack/bert-base-uncased-rotten-tomatoes', 
    'albert': 'textattack/albert-base-v2-rotten-tomatoes', 
    'distilbert': 'textattack/distilbert-base-uncased-rotten-tomatoes', 
    'roberta': 'textattack/roberta-base-rotten-tomatoes', 
    'xlnet': 'textattack/xlnet-base-cased-rotten-tomatoes', 
    
}

In [5]:
m1 = load_model(rotten_tomatoes_models['albert'])
m2 = load_model(rotten_tomatoes_models['distilbert'])
m3 = load_model(rotten_tomatoes_models['roberta'])
m4 = load_model(rotten_tomatoes_models['xlnet'])

# Models to be used as oracle
models = [m1, m2, m3, m4]
# Target model
model = load_model(rotten_tomatoes_models['bert'])

Loading model textattack/albert-base-v2-rotten-tomatoes...
Loading model textattack/distilbert-base-uncased-rotten-tomatoes...
Loading model textattack/roberta-base-rotten-tomatoes...


Some weights of the model checkpoint at textattack/roberta-base-rotten-tomatoes were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading model textattack/xlnet-base-cased-rotten-tomatoes...
Loading model textattack/bert-base-uncased-rotten-tomatoes...


# Gerando os templates
O método de rankeamento das palavras usado no PosNegTemplateGenerator é o Replace-1 Score

In [6]:
from template_generator.tasks.sentiment_analisys import PosNegTemplateGeneratorApp1

tg = PosNegTemplateGeneratorApp1(model, models)

### Número inicial de instâncias: 5

In [7]:
# Sampling instances
np.random.seed(220)
n_instances = 5
df_sampled = df.sample(n_instances)

instances = [x for x in df_sampled['text'].values]
instances

['the rules of attraction gets us too drunk on the party favors to sober us up with the transparent attempts at moralizing .',
 "cherry orchard is badly edited , often awkwardly directed and suffers from the addition of a wholly unnecessary pre-credit sequence designed to give some of the characters a 'back story . '",
 "there is more than one joke about putting the toilet seat down . and that should tell you everything you need to know about all the queen's men .",
 "the high-concept scenario soon proves preposterous , the acting is robotically italicized , and truth-in-advertising hounds take note : there's very little hustling on view .",
 "it may not be particularly innovative , but the film's crisp , unaffected style and air of gentle longing make it unexpectedly rewarding ."]

In [8]:
templates = tg.generate_templates(instances)

Ranking words using Replace-1 Score...


/tmp/ipykernel_6738/4227237112.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = softmax(tensor_logits[0]).detach().numpy()


Converting texts to sentences...
:: 5 sentences were generated.
Filtering instances by contaning ranked words...
:: 0 sentences remaining.
Filtering instances by relevant words...
:: 0 sentences remaining.
Predicting inputs...
:: Sentence predictions done.


#### Tempo de execução para 5 instâncias: 6m 8.6s
filipe: 2m 20.5s

In [9]:
tg.to_dataframe()

,label,original_text,masked_text,template_text


In [10]:
tg.lexicons

{'pos_verb': [], 'neg_verb': [], 'pos_adj': [], 'neg_adj': []}

### Número inicial de instâncias: 100

In [11]:
# Using all 100 instances
instances = [x for x in df['text'].values]

In [12]:
%%time
# 1m 38.7s (com outros processos pesados concorrendo, isso opde ter diminuido)
tg = PosNegTemplateGeneratorApp1(model, models)
templates = tg.generate_templates(instances)

Ranking words using Replace-1 Score...


/tmp/ipykernel_6738/4227237112.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = softmax(tensor_logits[0]).detach().numpy()


Converting texts to sentences...
:: 103 sentences were generated.
Filtering instances by contaning ranked words...
:: 3 sentences remaining.
Filtering instances by relevant words...
:: 0 sentences remaining.
Predicting inputs...
:: Sentence predictions done.
CPU times: user 14min 44s, sys: 1.75 s, total: 14min 46s
Wall time: 1min 38s


#### Tempo de execução para 100 instâncias: 1m 0.7s
filipe: 1m 0.7s

In [13]:
df_templates = tg.to_dataframe()
df_templates.insert(0, "template_index", df_templates.index.map(lambda x: int(x)+1))
df_templates

,template_index,label,original_text,masked_text,template_text


In [14]:
df_templates.to_csv("generated_templates/generated_templates_approach1.csv")

In [15]:
tg.lexicons

{'pos_verb': [], 'neg_verb': [], 'pos_adj': [], 'neg_adj': []}

# Usando os templates gerados pelo TemplateGenerator no CheckList

In [16]:
from checklist.editor import Editor
from checklist.test_suite import TestSuite
from checklist.test_types import MFT

In [17]:
lexicons = tg.lexicons
templates = tg.template_texts
masked = tg.masked_texts
labels = [sent.prediction.label for sent in tg.sentences]

editor = Editor()
editor.add_lexicon('pos_verb', lexicons['pos_verb'])
editor.add_lexicon('neg_verb', lexicons['neg_verb'])
editor.add_lexicon('pos_adj', lexicons['pos_adj'])
editor.add_lexicon('neg_adj', lexicons['neg_adj'])

suite = TestSuite()

In [18]:
for template, label, i in zip(templates, labels, range(len(templates))):
    t = editor.template(template, remove_duplicates=True, labels=int(label))

    suite.add(MFT(
        data=t.data,
        labels=label,
        capability="Vocabullary", 
        name=f"Test: MFT with vocabullary - template{i+1}",
        description="Checking if the model can handle vocabullary"))

In [19]:
suite.run(model.predict, overwrite=True)
suite.save('./suites/posneg-approach1.suite')

# Carregando suite de teste

In [20]:
from checklist.test_suite import TestSuite
suite = TestSuite.from_file('./suites/posneg-approach1.suite')

# suite.visual_summary_table()

In [22]:
passed = 0
failed = 0
for test_name in suite.tests:
    table = suite.visual_summary_by_test(test_name)
    
    failed += table.stats['nfailed']    
    passed += table.stats['npassed']
    assert table.stats['nfailed'] + table.stats['npassed'] == len(table.filtered_testcases)

print(f"{failed = } ({(failed/(passed+failed))*100:.2f}%)")
print(f"{passed = } ({(passed/(passed+failed))*100:.2f}%)")
print(f"total = {passed+failed}")
print("templates:", len(suite.tests))



ZeroDivisionError: division by zero

In [24]:
table.candidate_testcases

[{'examples': [{'new': {'text': "and that should need you everything you need to know about all the queen 's men .",
     'pred': '0',
     'conf': 0.6925088,
     'tokens': [['and',
       'that',
       'should',
       'need',
       'you',
       'everything',
       'you',
       'need',
       'to',
       'know',
       'about',
       'all',
       'the',
       'queen',
       "'s",
       'men',
       '.']]},
    'old': None,
    'label': 1,
    'succeed': 0}],
  'succeed': 0,
  'tags': []}]

In [25]:
table.filtered_testcases

[{'examples': [{'new': {'text': "and that should tell you everything you tell to know about all the queen 's men .",
     'pred': '1',
     'conf': 0.9949285,
     'tokens': [['and',
       'that',
       'should',
       'tell',
       'you',
       'everything',
       'you',
       'tell',
       'to',
       'know',
       'about',
       'all',
       'the',
       'queen',
       "'s",
       'men',
       '.']]},
    'old': None,
    'label': 1,
    'succeed': 1}],
  'succeed': 1,
  'tags': []},
 {'examples': [{'new': {'text': "and that should need you everything you need to know about all the queen 's men .",
     'pred': '0',
     'conf': 0.6925088,
     'tokens': [['and',
       'that',
       'should',
       'need',
       'you',
       'everything',
       'you',
       'need',
       'to',
       'know',
       'about',
       'all',
       'the',
       'queen',
       "'s",
       'men',
       '.']]},
    'old': None,
    'label': 1,
    'succeed': 0}],
  'succeed': 0,

In [28]:
table = suite.visual_summary_by_test('Test: MFT with vocabullary - template1')

failed = table.candidate_testcases
tests = table.filtered_testcases

for item in tests:
    # if not item in failed:
    print(item['examples'][0])

{'new': {'text': "and that should tell you everything you tell to know about all the queen 's men .", 'pred': '1', 'conf': 0.9949285, 'tokens': [['and', 'that', 'should', 'tell', 'you', 'everything', 'you', 'tell', 'to', 'know', 'about', 'all', 'the', 'queen', "'s", 'men', '.']]}, 'old': None, 'label': 1, 'succeed': 1}
{'new': {'text': "and that should need you everything you need to know about all the queen 's men .", 'pred': '0', 'conf': 0.6925088, 'tokens': [['and', 'that', 'should', 'need', 'you', 'everything', 'you', 'need', 'to', 'know', 'about', 'all', 'the', 'queen', "'s", 'men', '.']]}, 'old': None, 'label': 1, 'succeed': 0}


In [14]:
len(table.candidate_testcases)

1

In [7]:
dir(table)

['__annotations__',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_active_widgets',
 '_add_notifiers',
 '_all_trait_default_generators',
 '_call_widget_constructed',
 '_comm_changed',
 '_compare',
 '_control_comm',
 '_cross_validation_lock',
 '_default_keys',
 '_descriptors',
 '_dom_classes',
 '_gen_repr_from_keys',
 '_get_embed_state',
 '_get_trait_default_generator',
 '_handle_control_comm_msg',
 '_handle_custom_msg',
 '_handle_msg',
 '_holding_sync',
 '_instance_inits',
 '_is_numpy',
 '_lock_property',
 '_log_default',
 '_model_id',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_msg_callbacks',
 '_noti

In [19]:
table.tokenize_testcases

{'npassed': 1, 'nfailed': 1, 'nfiltered': 0}